In [1]:
# 나무위키 데이터셋 실험
from auto_preprocessing import word_preprocessing
from auto_preprocessing import autoencode
from auto_preprocessing import autocheck
import pickle
import os
from sentence_transformers import SentenceTransformer
from tqdm import tqdm
import os, pickle


data_dict = {}

# 모든 파일 읽기
for root, dirs, files in os.walk("../data/namu"):
    for file in files:
        with open(os.path.join(root, file), "r", encoding="utf-8") as f:
            data = f.read()
            data_dict[file] = data

In [2]:
# 단어 슬라이드 조절

slides = [ 1,3,5,7,9 ]
for slide in slides:
    result_dict = word_preprocessing(data_dict, slide=slide)
    with open(f"./result/preprocess/slide_{slide}.pkl", "wb") as f:
        pickle.dump(result_dict, f)

In [3]:
autoencode()

Encoding: 100%|███████████████████████████████████████████████████████| 7/7 [00:10<00:00,  1.55s/it]


In [4]:
autocheck()

Checking: 100%|███████████████████████████████████████████████████████| 7/7 [00:01<00:00,  5.42it/s]


In [5]:
# 문장별 실험

import re
import pickle

def sentence_preprocessing(data_dict):
    result_dict = {}

    for doc in data_dict.keys():
        result_dict[doc] = []

        text = data_dict[doc].replace('\n', ' ')

        # . 또는 ? 뒤에 공백이 오는 패턴으로 문장 분리
        sentences = re.split(r'(?<=[.?])\s+', text)

        result_dict[doc] = sentences

    with open(f"./result/preprocess/sentence.pkl", "wb") as f:
        pickle.dump(result_dict, f)

    return result_dict


In [6]:
preprocessed_dict = sentence_preprocessing(data_dict)
print(preprocessed_dict)

{'고대-1.txt': ['르네상스 시대에 자신들과 중세를 구분하면서 시작되었으며, 카를 마르크스의 역사 성장 단계 이론이 나온 후, 경제적 개진 수준에 따라서 분간하는 추세가 우월하다.', '사실 각 지역 간에 정형화된 특징의 고대나 중세란 실재하지 않는다.', '한편 마르크스의 세기 구분론의 규격으로 보자면 노예 노동이 경제의 기본을 이루는 시대를 말하며 많은 사회경제사 고찰에서 아직도 이 관념을 받아들인다.', '보수적 사학자인 이영훈이 조선을 고대 노예제 사회로 규제한 것이 그 예시.', '이에 대한 곡해로 조선 사회는 노예의 수가 적었다거나, 노예 경제는 세계사를 기준으로 감정한다면 오히려 예외적인 경우라 반대하기도 하지만, 여기서의 노예 현황은 명시적 신분이라기보다는 토지와 맺는 관계를 의미하는 것에 가깝다.', '겉모습상 평민이라도 농지와의 관계보다 지배자와의 관계가 큰 비중을 차지한다면 노예라 볼 수 있다.', '마르크스의 시대 구분론은 아직도 경제사 연구에서는 널리 유통되며, 많은 학자들이 생산수단과 생산자와의 관련을 토대로 고대, 중세, 근대의 구별을 사용하고 있다.', '담여로 유사 시대의 시작부터 기원 후 한참 후까지의 정말 긴 시간이 고대로 묶이다 보니, 지금 사람들이 보기에는 넉넉히 옛날 옛적의 고대인들이 당시 기준으로 더 옛날이었던 시대를 신비스러운 고대로 판단하곤 했다.', '사료를 보면 로마인들이 이집트 유적을 불가사의한 고대 관광지로 간주한다던지, 후한 말엽 시기의 사람들이 전국칠웅과 진의 자취를 신비스럽게 여긴다든지 하는 것들을 쉽게 찾아볼 수 있다.', '한국사에서는 고조선부터 삼국통일까지나 남북국시대까지, 또는 후삼국시대까지를 고대로 보는 편이다.', '이는 노예 노동이 국가 경제의 주가 되어야 한다는 마르크스의 유물 역사관에 따른 시대 구분론에서 벗어나 있다는 감정을 받고 있다.', '애초에 마르크스의 시대 구분론은 원시 공산주의-노예제 사회-봉건주의-자본주의 순이다.', '유럽에만 적용 가능한 무의미한 시대 구분으로 실

In [7]:
def sentence_encode(model=SentenceTransformer("jhgan/ko-sbert-sts")):

    with open("./result/preprocess/sentence.pkl", "rb") as f:
        result_dict = pickle.load(f)

    encoded_dict = {}
    
    for doc in result_dict.keys():
        sentences = result_dict[doc]
        embeddings = model.encode(sentences, normalize_embeddings=True)
        encoded_dict[doc] = embeddings

    with open("./result/encode/sentence.pkl", "wb") as f:
        pickle.dump(encoded_dict, f)
    
    return encoded_dict

In [8]:
encoded_dict = sentence_encode()

In [9]:
import numpy as np
import pandas as pd
import torch

def sentence_check(model=SentenceTransformer("jhgan/ko-sbert-sts"), threshold1 = 0.85, threshold2 = 0.10):

    with open("./result/encode/sentence.pkl", "rb") as f:
        result_dict = pickle.load(f)
    
    files = list(result_dict.keys())

    for i in tqdm(range(len(files))):
        pivot = np.array(result_dict[files[i]])
        pivot = torch.tensor(pivot, dtype=torch.float32, device="cuda")

        for j in range(i+1, len(files)):
            target = np.array(result_dict[files[j]])
            target = torch.tensor(target, dtype=torch.float32, device="cuda")

            sim_matrix = torch.mm(pivot, target.T)
            cnt = (sim_matrix >= threshold1).sum(dim=1).gt(0).sum().item()
            label = 1 if cnt >= len(pivot) * threshold2 else 0

            row = pd.DataFrame([{
                "file1": files[i],
                "file2": files[j],
                "similarity": cnt / len(pivot) * 100,
                "label": label
            }])

            row.to_csv("./result/check/sentence.csv", mode='a', header=False, index=False)
                                                                

In [10]:
sentence_check()

100%|██████████| 18/18 [00:00<00:00, 77.39it/s]
